# Music Training notebook

* This notebook uses a model consisting of LSTM (Long Short Term Memory) networks to learn musical notes from multiple files in order to predict the most probable note to be played next. 

In [1]:
import glob
import pickle
import numpy
import IPython
import random
import pygame
from music21 import midi, converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Playing .mid (pronounced 'mid-ee') music files

Mention the path to 3 files:
* input_file_loc: Location of the input musical files
* notes_loc: Location where the extracted notes will get stored
* weights_loc: Location to store the weights of the trained model

In [2]:
genre = 'Rock'
input_file_loc = 'songs/' + genre +'/'
notes_loc = 'notes/notes_' + genre
weights_loc = 'weights/weights_' + genre + '.hdf5'

In [3]:
list_songs = glob.glob(input_file_loc + '*.mid')
num_songs = len(list_songs)
print('Number of songs used in the training: {}'.format(num_songs))
print('let\'s hear one of the songs used in the training..')
song_num = random.randint(0, num_songs-1)
print('Playing {}.....'.format(list_songs[song_num]))
mf = midi.MidiFile()
mf.open(list_songs[song_num])
mf.read()
mf.close()
streamMidi = midi.translate.midiFileToStream(mf)
streamMidi.show('midi')

Number of songs used in the training: 1
let's hear one of the songs used in the training..
Playing songs/Rock/Guns n Roses - Sweet Child O Mine.mid.....


## Extraction of notes from the core musical components

In [4]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
#     print('Number of files: {}'.format(len(glob.glob("songs/*.mid"))))
    for file in glob.glob(input_file_loc + "*.mid"):
        midi = converter.parse(file)
        
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open(notes_loc, 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [5]:
notes = get_notes()
print(len(notes))

Parsing songs/Rock/Guns n Roses - Sweet Child O Mine.mid
3068


## Preparing the input and output sequences for the model 

In [6]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [7]:
n_vocab = len(set(notes))
print('Number of notes: {}'.format(n_vocab))
network_input, network_output = prepare_sequences(notes, n_vocab)

Number of notes: 73


## Forming the model

In [8]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.load_weights(weights_loc)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [9]:
model = create_network(network_input, n_vocab)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
activation (Activation)      (None, 256)               0

In [10]:
filepath = weights_loc
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

## Start your training, it will be tough, but you'll get over it :-)

In [ ]:
 model.fit(network_input, network_output, epochs=2, callbacks=callbacks_list)

Epoch 1/2
34/34 [==============================] - 92s 3s/step - loss: 0.1653
Epoch 2/2
 9/34 [======>.......................] - ETA: 1:07 - loss: 0.0779